In [2]:
import json
import os
from numpy import dot
from numpy.linalg import norm

In [3]:
with open("vietnamese-stopwords.txt",'r') as f:
    stopwords = f.read()


In [4]:
# print(stopwords)

In [5]:
def get_vector_roberta(sens):
    # Encode using roberta class
    tokens_ids = roberta.encode(sens)
    # assert tokens_ids.tolist() == [0, 451, 71, 3401, 1384, 168, 234, 5, 2]
    # Extracted feature using roberta model
    tokens_embed = roberta.extract_features(tokens_ids)
    # assert tokens_embed.shape == (1, 9, 512)
    return tokens_embed,tokens_embed[0,-1,:]

In [23]:
tokens_embed

NameError: name 'tokens_embed' is not defined

In [6]:
data_path = "./data/"

In [7]:
with open(os.path.join(data_path,"legal_corpus.json"),'r') as f:
    law_data = json.load(f)

In [8]:
count_doc = 0
for item in law_data:
    count_doc += len(item['articles'])
print(count_doc)


61425


In [9]:
print(law_data[0]['law_id'])

01/2009/tt-bnn


In [10]:
print(law_data[0]['articles'][0])

{'article_id': '1', 'title': 'Điều 1. Phạm vi áp dụng', 'text': 'Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.'}


In [11]:
title_corpus = []
articles_corpus = []

In [12]:
from underthesea import word_tokenize

In [13]:
for law in law_data:
    for article in law['articles']:
#         title_corpus.append(filter_stopword(article['title'],stopwords))
#         articles_corpus.append(filter_stopword(article['text'],stopwords))
        title_corpus.append(word_tokenize(article['title'], format="text"))
        articles_corpus.append(word_tokenize(article['text'], format="text"))

In [14]:
law_data_dict = {doc['law_id']:doc['articles'] for doc in law_data}

In [15]:
def get_law_id_from_title(title):
    lst_law = []
    for item in law_data_dict.keys():
        for it in law_data_dict[item]:
            if it['title'] == title:
                lst_law.append(item)
                break
    return lst_law

In [16]:
print(len(title_corpus))
print(len(articles_corpus))

61425
61425


In [17]:
def filter_stopword(sen,stopwords):
    text = ' '.join([word for word in sen.split() if word not in stopwords])
    return text

In [18]:
from rank_bm25 import BM25Okapi

tokenized_title = [doc.split(" ") for doc in title_corpus]
tokenized_articles = [doc.split(" ") for doc in articles_corpus]

bm25_title = BM25Okapi(tokenized_title)
bm25_article = BM25Okapi(tokenized_articles)

In [19]:
with open(os.path.join(data_path,"train_question_answer.json"),'r') as f:
    train_data = json.load(f)

In [20]:
print(type(train_data))

<class 'dict'>


In [21]:
print(train_data['_count_'])

3196


In [22]:
train_data.keys()

dict_keys(['_name_', '_date_time_', '_count_', 'items'])

In [23]:
for ind,question in enumerate(train_data['items']):
    if ind == 3:
        print(question)
        break

{'question_id': '3aed91309b72097b34427dd28bebd98a', 'question': 'Tử tù chết sau khi tiêm thuốc độc mà không có thân nhân nhận xác thì được hỗ trợ mai táng như thế nào?', 'relevant_articles': [{'law_id': '53/2010/qh12', 'article_id': '60'}, {'law_id': '82/2011/nđ-cp', 'article_id': '9'}]}


In [24]:
def get_doc_by_law_and_article(relevant_articles):
    lst_doc = []
    for law in relevant_articles:
        tmp = law_data_dict[law['law_id']]
        for item in tmp:
            if item['article_id'] == law['article_id']:
                lst_doc.append({item['title']:item['text']})
    return lst_doc

In [25]:
get_doc_by_law_and_article([{'law_id': '53/2010/qh12', 'article_id': '60'}])

[{'Điều 60. Giải quyết việc xin nhận tử thi, hài cốt của người bị thi hành án tử hình': '1. Việc giải quyết nhận tử thi được thực hiện như sau:\na) Trước khi thi hành án tử hình, thân nhân hoặc người đại diện hợp pháp của người chấp hành án được làm đơn có xác nhận của Ủy ban nhân dân cấp xã nơi cư trú gửi Chánh án Tòa án đã xét xử sơ thẩm đề nghị giải quyết cho nhận tử thi của người chấp hành án để an táng; trường hợp người chấp hành án là người nước ngoài thì đơn phải có xác nhận của cơ quan có thẩm quyền hoặc cơ quan đại diện ngoại giao tại Việt Nam của nước mà người chấp hành án mang quốc tịch và phải được dịch ra tiếng Việt. Đơn phải ghi rõ họ tên, địa chỉ người nhận tử thi, quan hệ với người chấp hành án; cam kết bảo đảm yêu cầu về an ninh, trật tự, vệ sinh môi trường và tự chịu chi phí;\nb) Chánh án Tòa án đã xét xử sơ thẩm thông báo bằng văn bản cho người có đơn đề nghị về việc cho nhận tử thi hoặc không cho nhận tử thi khi có căn cứ cho rằng việc nhận tử thi ảnh hưởng đến an n

In [28]:
query = 'Tử tù chết sau khi tiêm thuốc độc mà không có thân nhân nhận xác thì được hỗ trợ mai táng như thế nào?'
query = word_tokenize(query,format='text')
#query = filter_stopword(query,stopwords)
print(query)
tokenized_query = query.split(' ')
res_articles = bm25_article.get_top_n(tokenized_query, articles_corpus, n=10)
res_title = bm25_title.get_top_n(tokenized_query, title_corpus, n=1000)

Tử_tù chết sau khi tiêm thuốc_độc mà không có thân_nhân nhận xác thì được hỗ_trợ mai_táng như thế_nào ?


In [32]:
print(res_articles[:2])

['1 . Trường_hợp thân_nhân của người hiến bộ_phận cơ_thể người sau khi chết , hiến xác có nhu_cầu tổ_chức tang_lễ và mai_táng di_hài cho người hiến bộ_phận cơ_thể người sau khi chết , hiến xác được hỗ_trợ mai_táng_phí bằng 10 tháng lương cơ_sở . Thân_nhân của người hiến bộ_phận cơ_thể người sau khi chết , hiến xác phải xuất_trình với cơ_sở y_tế , cơ_sở tiếp_nhận và bảo_quản xác của người hiến giấy_tờ chứng_minh là thân_nhân của người đã hiến bộ_phận cơ_thể người sau khi chết , hiến xác để nhận chế_độ tổ_chức tang_lễ và mai_táng di_hài theo quy_định tại Khoản này . 2 . Trường_hợp cơ_sở y_tế , cơ_sở tiếp_nhận và bảo_quản xác của người hiến tổ_chức tang_lễ và mai_táng được thanh_toán chi_phí theo thực_tế phát_sinh nhưng tối_đa không quá 10 tháng lương cơ_sở .', '1 . Người đang hưởng lương hưu trước ngày 01 tháng 01 năm 2016 mà chết từ ngày 01 tháng 01 năm 2016 trở_đi thì người lo mai_táng được nhận một lần trợ_cấp mai_táng bằng 10 lần mức lương cơ_sở_tại tháng người đang hưởng lương hưu c

In [61]:
doc_scores = bm25_article.get_scores(tokenized_query)
print((doc_scores[:10]))

[ 4.08005858 11.58885014  3.88619918  7.95027111  8.16011716 13.88503808
 12.72734876  7.96136732  5.63017382 12.30007837]


In [44]:
lst_law_id_relevant = []
for item in res_title:
    #print("-"*50)
    #result = " ".join(item)
    lst_law_id_relevant += get_law_id_from_title(item)

lst_law_id_relevant = list(set(lst_law_id_relevant))
print(lst_law_id_relevant)

['176/2013/nđ-cp', '53/2010/qh12', '22/2018/nđ-cp', '105/2016/ttlt-bqp-bca-blđtbxh', '52/2014/qh13', '98/2018/nđ-cp', '104/2017/tt-btc', '39/2015/nđ-cp', '56/2017/nđ-cp', '02/2015/ttlt-bca-bqp-byt', '31/2013/nđ-cp', '102/2014/nđ-cp', '74/2015/nđ-cp', '57/2018/nđ-cp', '25/2014/ttlt-byt-btc', '49/2010/qh12', '41/2019/qh14', '03/2017/tt-btnmt', '21/2019/tt-bct', '39/2009/qh12', '21/2012/qđ-ttg', '14/2016/tt-byt', '112/2017/nđ-cp', '20/2012/tt-bkhcn', '108/2016/qh13', '27/2013/ttlt-bgdđt-btc', '38/2013/qh13', '20/2021/nđ-cp', '12/2017/tt-blđtbxh', '33/2016/nđ-cp', '45/2014/tt-btc', '85/2010/qđ-ttg', '102/2018/nđ-cp', '105/2012/nđ-cp', '117/2020/nđ-cp', '56/2017/tt-byt', '43/2020/nđ-cp', '15/2017/tt-btc', '111/2010/nđ-cp', '82/2011/nđ-cp', '73/2017/tt-btc', '07/2019/tt-btp', '16/2014/ttlt-btp-btnmt-nhnn', '04/2018/ttlt-bca-bqp-tandtc-vksndtc', '18/2019/nđ-cp', '04/2020/tt-btp', '29/2018/tt-btc', '80/2011/nđ-cp', '91/2015/qh13', '65/2018/nđ-cp', '19/2015/nđ-cp', '32/2018/tt-blđtbxh', '02/201

In [45]:
get_law_id_from_title("Điều 14. Thẩm quyền xử phạt của Công an nhân dân")

['55/2018/nđ-cp']

In [46]:
from tqdm import tqdm
import pandas as pd
def get_similar(query,lst_law):
    _, query_vector = get_vector_roberta(query)
    query_vector = query_vector.detach()
    law_id = []
    article_id = []
    score = []
    for item in lst_law:
        docs = law_data_dict[item]
        for it in tqdm(docs):
            _,last_res = get_vector_roberta(it['text'][:512])
            last_res = last_res.detach()
            law_id.append(item)
            article_id.append(it['article_id'])
            score.append(cosine(query_vector,last_res))
    df = pd.DataFrame({"law_id":law_id,"article_id":article_id,"score":score})
    df = df.sort_values(by=['score'])
    return df

In [47]:
def cosine(a,b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

In [49]:
# df = get_similar(query,["53/2010/qh12"])

In [161]:
df = df.reset_index(drop=True)
# df.tail(20)

In [50]:
max_relevant = 0
max_question = ""
for ind,question in enumerate(train_data['items']):
    if len(question['relevant_articles']) > max_relevant:
        max_relevant = len(question['relevant_articles'])
        max_question = question

In [51]:
max_relevant,max_question

(3,
 {'question_id': '8566fb9669f3c2086b9e5cea4306a162',
  'question': 'Cục Xuất bản, In và Phát hành có trách nhiệm gì trong việc thẩm định đề án và xác nhận đăng ký hoạt động xuất bản, phát hành',
  'relevant_articles': [{'law_id': '195/2013/nđ-cp', 'article_id': '17'},
   {'law_id': '01/2020/tt-btttt', 'article_id': '24'},
   {'law_id': '195/2013/nđ-cp', 'article_id': '18'}]})

In [169]:

# for i in [5,10,15,20,25]:
#     find = 0
#     total = 0
#     for ind,question in enumerate(train_data['items']):
#         query = question['question']
#         tokenized_query = query.split(' ')
#         res_title = bm25_title.get_top_n(tokenized_query, title_corpus, n=i)
#         lst_law_id_relevant = []
#         for item in res_title:
#             lst_law_id_relevant += get_law_id_from_title(item)
#         count = 0
#         for item in question['relevant_articles']:
#             if item['law_id'] in lst_law_id_relevant:
#                 find += 1
#         total += len(question['relevant_articles'])
#     print("n = {} cover {}".format(i,1.0*find/total))

n = 5 cover 0.711340206185567
n = 10 cover 0.7832019405700424
n = 15 cover 0.8101879927228623
n = 20 cover 0.8302001212856277
n = 25 cover 0.8441479684657368


In [52]:
for ind,question in enumerate(train_data['items']):
    if ind == 3:
        print(question)
        break

{'question_id': '3aed91309b72097b34427dd28bebd98a', 'question': 'Tử tù chết sau khi tiêm thuốc độc mà không có thân nhân nhận xác thì được hỗ trợ mai táng như thế nào?', 'relevant_articles': [{'law_id': '53/2010/qh12', 'article_id': '60'}, {'law_id': '82/2011/nđ-cp', 'article_id': '9'}]}


In [83]:
import random
def generate_training_data(question,num_generator=50):
    list_answer = get_doc_by_law_and_article(question['relevant_articles'])
    list_answer = [list(x.values())[0] for x in list_answer]
    random_list = random.sample(articles_corpus,num_generator)
    lst_result = []
    for item in list_answer:
        list_generate = [question['question']+"\t"+item+"\t"+x for x in random_list]
        lst_result += list_generate
    return lst_result



In [85]:
g[0]

'Tử tù chết sau khi tiêm thuốc độc mà không có thân nhân nhận xác thì được hỗ trợ mai táng như thế nào?\t1. Việc giải quyết nhận tử thi được thực hiện như sau:\na) Trước khi thi hành án tử hình, thân nhân hoặc người đại diện hợp pháp của người chấp hành án được làm đơn có xác nhận của Ủy ban nhân dân cấp xã nơi cư trú gửi Chánh án Tòa án đã xét xử sơ thẩm đề nghị giải quyết cho nhận tử thi của người chấp hành án để an táng; trường hợp người chấp hành án là người nước ngoài thì đơn phải có xác nhận của cơ quan có thẩm quyền hoặc cơ quan đại diện ngoại giao tại Việt Nam của nước mà người chấp hành án mang quốc tịch và phải được dịch ra tiếng Việt. Đơn phải ghi rõ họ tên, địa chỉ người nhận tử thi, quan hệ với người chấp hành án; cam kết bảo đảm yêu cầu về an ninh, trật tự, vệ sinh môi trường và tự chịu chi phí;\nb) Chánh án Tòa án đã xét xử sơ thẩm thông báo bằng văn bản cho người có đơn đề nghị về việc cho nhận tử thi hoặc không cho nhận tử thi khi có căn cứ cho rằng việc nhận tử thi ản

In [87]:
import torch
from transformers import AutoModel, AutoTokenizer
from torch import nn as nn


class FineTunePhobert(nn.Module):
    def __init__(self,n_output=1):
        super(FineTunePhobert,self).__init__()
        self.linear = nn.Linear(768,n_output)
        self.phobert = AutoModel.from_pretrained("vinai/phobert-base")
        self.tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
    def forward(self,x):
        x = torch.tensor([tokenizer.encode(x)])

AttributeError: 'tuple' object has no attribute 'shape'